In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import metrics
from sklearn.linear_model import Lasso
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import (
    LinearRegression,
    LassoCV,
    RidgeCV,
)
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# grid search hyperparameters for lasso regression
from numpy import arange
from pandas import read_csv
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso

In [2]:
df = pd.read_csv('Test_drug_prediction.csv',index_col=0)

In [3]:
df.head()

,target,LOC729737,CICP3,LOC101060494,LINC00115,LINC01128,LOC284600,LOC100130417,SAMD11,NOC2L,...,MAFIP,LOC283788,KIR3DS1,LOC100507412,KIR2DS5,LOC100288966,KIR2DS1,MGC70870,HLA.DRB4,HLA.DRB3
1,2.646123,8.616,6.249,5.054,2.287,6.513,7.131,5.631,6.570,8.032,...,6.635,3.530,2.286,8.587,2.428,6.242,2.186,6.282,2.641,4.487
2,2.510278,8.876,6.623,5.462,2.446,7.144,6.936,6.677,7.317,8.142,...,6.185,3.158,2.286,8.106,2.400,6.342,2.141,5.654,2.541,5.107
3,1.612207,9.265,6.461,6.320,2.288,7.004,7.306,6.100,7.144,8.794,...,6.337,2.797,2.286,9.080,2.286,6.242,2.286,5.968,2.616,4.057
4,1.517873,9.371,6.227,6.728,2.287,7.244,7.329,5.535,6.379,8.151,...,6.028,3.333,2.286,8.219,2.408,6.394,2.144,8.205,2.452,4.458
5,1.200902,8.413,6.233,3.416,2.291,6.554,7.438,6.160,7.233,8.042,...,6.065,3.235,2.286,8.715,2.429,6.708,2.184,5.825,2.612,4.517


In [4]:
# check the shape of dataframe
df.shape

(51, 23060)

In [5]:

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 1 to 51
Columns: 23060 entries, target to HLA.DRB3
dtypes: float64(23060)
memory usage: 9.0 MB


In [6]:
df.isnull().sum()

target          0
LOC729737       0
CICP3           0
LOC101060494    0
LINC00115       0
               ..
LOC100288966    0
KIR2DS1         0
MGC70870        0
HLA.DRB4        0
HLA.DRB3        0
Length: 23060, dtype: int64

In [7]:
df.head()

,target,LOC729737,CICP3,LOC101060494,LINC00115,LINC01128,LOC284600,LOC100130417,SAMD11,NOC2L,...,MAFIP,LOC283788,KIR3DS1,LOC100507412,KIR2DS5,LOC100288966,KIR2DS1,MGC70870,HLA.DRB4,HLA.DRB3
1,2.646123,8.616,6.249,5.054,2.287,6.513,7.131,5.631,6.570,8.032,...,6.635,3.530,2.286,8.587,2.428,6.242,2.186,6.282,2.641,4.487
2,2.510278,8.876,6.623,5.462,2.446,7.144,6.936,6.677,7.317,8.142,...,6.185,3.158,2.286,8.106,2.400,6.342,2.141,5.654,2.541,5.107
3,1.612207,9.265,6.461,6.320,2.288,7.004,7.306,6.100,7.144,8.794,...,6.337,2.797,2.286,9.080,2.286,6.242,2.286,5.968,2.616,4.057
4,1.517873,9.371,6.227,6.728,2.287,7.244,7.329,5.535,6.379,8.151,...,6.028,3.333,2.286,8.219,2.408,6.394,2.144,8.205,2.452,4.458
5,1.200902,8.413,6.233,3.416,2.291,6.554,7.438,6.160,7.233,8.042,...,6.065,3.235,2.286,8.715,2.429,6.708,2.184,5.825,2.612,4.517


In [8]:
X = df.drop(['target'],axis=1)
y = df.target

In [9]:
X.shape

(51, 23059)

### correlation

In [10]:
cor_matrix = X.corr().abs()
print(cor_matrix)

              LOC729737     CICP3  LOC101060494  LINC00115  LINC01128  \
LOC729737      1.000000  0.134614      0.625309   0.222565   0.045508   
CICP3          0.134614  1.000000      0.285111   0.209573   0.003037   
LOC101060494   0.625309  0.285111      1.000000   0.026843   0.060985   
LINC00115      0.222565  0.209573      0.026843   1.000000   0.158938   
LINC01128      0.045508  0.003037      0.060985   0.158938   1.000000   
...                 ...       ...           ...        ...        ...   
LOC100288966   0.020113  0.330117      0.036669   0.128457   0.046354   
KIR2DS1        0.034720  0.095736      0.067118   0.008014   0.157825   
MGC70870       0.246052  0.017499      0.130624   0.095651   0.108954   
HLA.DRB4       0.116352  0.297350      0.135662   0.214760   0.200612   
HLA.DRB3       0.017121  0.281227      0.122750   0.305722   0.123805   

              LOC284600  LOC100130417    SAMD11     NOC2L    KLHL17  ...  \
LOC729737      0.289070      0.160054  0.017127

In [11]:
# split the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                    shuffle=True, random_state=2)

In [12]:
rfr =  RandomForestRegressor()

rfr.fit(X_train,y_train)

y_pred = rfr.predict(X_test)


print("R2 Error :" ,r2_score (y_test,y_pred))

R2 Error : 0.10275135062638596


### LASSO

In [13]:
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score



In [14]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_scaled = sc.fit_transform(X)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.20, 
                                                    shuffle=True, random_state=2)

In [16]:
###yooo
features = X.columns
model_lasso = LassoCV(alphas = [1, 0.1, 0.001, 0.0005]).fit(X_train, y_train)
#rmse_cv(model_lasso).mean()
coef = pd.Series(model_lasso.coef_, index = features)
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

Lasso picked 135 variables and eliminated the other 22924 variables


C:\Users\kale_\anaconda3\envs\new_stream\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.013824024552264027, tolerance: 0.003475808477652696
  positive)


In [34]:
y_pred_lasso = model_lasso.predict(X_test)
print("R2 Error :" ,r2_score (y_test,y_pred_lasso))


R2 Error : 0.01830265146162069


In [35]:
Selected_feat = coef[coef!=0]
Selected_feat

LINC00337       0.026094
LOC100132942    0.063610
PLA2G2C         0.156274
MUL1           -0.008242
PFN1P10        -0.024065
                  ...   
IGLV3.25       -0.011779
TYMP            0.000367
PPP4R3CP       -0.000380
CXorf21         0.017118
LOC100287728    0.011754
Length: 135, dtype: float64

### the above are selected gene set less than 200 obtained by LASSO regression with r2 score of 0.018